# Instalações

In [57]:
import pandas as pd
import numpy as np
import re 
import string
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import spacy
import unicodedata
import emoji_data_python
import emoji
import nltk 
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
import pickle

In [2]:
#Baixando modelo de lematização
!python -m spacy download pt_core_news_lg

     -------------------------------------- 568.2/568.2 MB 4.3 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [ ]:
#importações e instalações para o bert
#recomenda-se reniciar o kernel e executar primeiro essas instalções, caso queira testar o Bert

# Executando tabela re-analisada e Pré-Processamento

## Limpeza de dados

In [3]:
#Base Refatorada, feita pelo projeto 4 https://github.com/2023M6T4-Inteli/Projeto4
df = pd.read_csv("C:\\Users\\Rodrigo.INTELI\\Downloads\\BASE-REFATORADA - BASE-REFATORADA.csv")
df = df[['texto', 'sentimento']]
df.head()

,texto,sentimento
0,Minuto touro de ouro,1
1,@ricktolledo Sim,0
2,Queria saber se a Btg banking é a própria btg ...,1
3,👏👏👏👏,1
4,Vamos ajudar o Brasil na principal categoria d...,1


In [4]:
#retirando valores nulos e duplicados da tabela 
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)

## Funções de Pré-Processamento

In [5]:
#Fução para retirar url dos textos
def remove_urls(text):
    """Berilgan matndan URL larini o'chiradi"""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [6]:
#Teste
remove_urls('https://www.youtube.com/results?search_query=streamlit+')

''

In [7]:
#Retirada de caracteres especias
def remover_caracteres_especiais(texto):
    texto_sem_acentos = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto_sem_especiais = re.sub('[^a-zA-Z0-9 \\\]', '', texto_sem_acentos)
    return texto_sem_especiais

In [8]:
#Teste
remover_caracteres_especiais('ç´Â')

'c A'

In [5]:
#Substituir os emojis por palavras em pt-br
def emoji(texto):
    # Substituição de emojis
    return emoji.demojize(texto, delimiters=(' ', ' '), language='pt')
     

In [6]:
#Teste
emoji('🚀')

AttributeError: 'function' object has no attribute 'demojize'

In [69]:
#lematização e lower
nlp = spacy.load("pt_core_news_lg")
def lema(texto):
    doc = nlp(texto.lower())
    text = []
    for word in doc:
        text.append(word.lemma_)
    return ' '.join(text)
    

In [71]:
#Teste
lema('comida andar matou soco')

'comida andar matar soco'

In [23]:
#Erros gramaticais e abreviações
#Dicionario de abreviações para substituir 
def substituir_abreviacoes(frase):
    dicionario_girias = {
    'vc': 'voce',
    'vcs':'voce',
    'Vc': 'voce',
    'pq': 'porque',
    'tbm': 'tambem',
    'q': 'que',
    'td': 'tudo',
    'blz': 'beleza',
    'flw': 'falou',
    'kd': 'cade',
    'to': 'estou',
    'mt': 'muito',
    'cmg': 'comigo',
    'ctz': 'certeza',
    'jah': 'ja',
    'naum': 'nao',
    'ta': 'esta',
    'eh': 'e',
    'vlw': 'valeu',
    'p': 'para',
    'qnd': 'quando',
    'msm': 'mesmo',
    'fzr': 'fazer',
    'agr': 'agora',
    'btgpactual': 'btg',
    'pactual': 'btg',
    }
    palavras = frase.split()
    for i, palavra in enumerate(palavras):
        if palavra in dicionario_girias:
            palavras[i] = dicionario_girias[palavra]
    return ' '.join(palavras)

In [27]:
#Teste
substituir_abreviacoes('vc fzr pactual')

'voce fazer btg'

In [48]:
#Retirada de stop-words e pontuações
def stop_punct(texto):
    stop_words = ['eu', 'tu', 'ele', 'ela', 'nós', 'nos', 'vós', 'vos', 'eles', 'elas', 'um', 'em', 'de'
                  'isso', 'isto', 'aquilo', 'algum', 'alguma', 'alguns', 'algumas',
                  'outro', 'outra', 'outros', 'outras', 'tão', 'tal', 'tanto', 'tanta', 'tantos', 'tantas', 
                  'seu', 'sua', 'seus', 'suas', 'dele', 'dela', 'deles', 'delas', 
                  'quem', 'qual', 'quais', 'que', 'onde', 'como', 'e','um', 'as', 'no',
                  'para', 'por', 'com', 'sem', 'sob', 'sobre', 'de', 'da', 'desde', 'em', 'entre', 'porque',
                  'á', 'a', 'o', 'ola', 'olá', 'pra', 'para', 'bemvindo', 'benvindo', 'bem-vindo', 'bemvindos', 'aqui', 'vai', 'na', 'no', 'esse', 'este', 'voce', 'nosso', 'ou', 'btg','ser', 'mais', 'ter', 'meu', 'se', 'esta', 'todo', 'estar']
    
    pontuacoes = ['.', ',', ';', ':', '!', '?', '-', '—', '_', '/', '\\', '(', ')', '[', ']', '{', '}', '"', "'", '`', '~', '^', '@', '#', '%', '&', '*']

    new = []
    for word in texto.split():
        if word not in stop_words and word not in pontuacoes:
            new.append(word)
    return ' '.join(new)
    

In [50]:
#Teste
stop_punct('andar eu , tu ola sem ! correr')

'andar correr'

In [36]:
#Removendo espaços extras
def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=str(text))
    return without_sc

In [41]:
#Teste

' '

In [38]:
#Função para remover @
def remove_arroba(texto):
    regex = r"@\w+"
    return re.sub(regex, "", texto)

In [40]:
#Teste
remove_arroba('@ro')

''

In [51]:
#Função pipeline 
def pipeline(texto):
    texto = remove_urls(texto)
    texto = remover_caracteres_especiais(texto)
    texto = remove_arroba(texto)
    texto = substituir_abreviacoes(texto)
    texto = emoji(texto)
    texto = stop_punct(texto)
    texto = lema(texto)
    #A lematização gera stop-words
    texto = stop_punct
    return texto

In [ ]:
#Teste

In [ ]:
#Aplicando as funções na tabela 

# Novas Features

## Número de tokens

# Análise Descritiva

## Balanceamento
- Gráfico para verificar balanceamento dos dados

## Word-Cloud Positivo
- Word-Cloud com palavras com labels positivas

## Word-Cloud Negativo
- Word-Cloud com palavras negativas

# DOC2VEC
- Métricas recall
- Acuracia de treino e teste para verificar overfitting
- Juntar com as novas features
- Algoritimos usados:

In [ ]:
#testes de resultado
#predict_proba (probabilidade das classes)
#predict (sentimento na frase)

# BERT
- Métricas recall
- Acurácia de treino e teste para verificar overfitting

In [ ]:
#testes de resultado
#predict_proba (probabilidade das classes)
#predict

# Baixando melhor modelo